# Agent

In [1]:
import mlflow
from dotenv import load_dotenv

load_dotenv()

# Optional: Set an experiment to organize your traces
mlflow.set_experiment("IsNex Tracing")

# Enable tracing
mlflow.langchain.autolog()  # type: ignore

2025/12/19 19:54:41 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/19 19:54:41 INFO mlflow.store.db.utils: Updating database tables
2025/12/19 19:54:41 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/19 19:54:41 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2025/12/19 19:54:41 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/19 19:54:41 INFO alembic.runtime.migration: Will assume non-transactional DDL.


In [2]:
from langchain.messages import HumanMessage, AIMessage, ToolMessage
from langchain.tools import tool
from src.core.utils.logger import get_logger

logger = get_logger(__name__)


@tool
def get_weather(location: str) -> str:
    """Get the weather for a location"""
    # Fake weather data
    return f"The weather in {location} is sunny and 25°C"


@tool
def calculate(expression: str) -> str:
    """Calculate a mathematical expression"""
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"


@tool
def search_web(query: str) -> str:
    """Search the web for information"""
    return f"Search results for '{query}': Found 10 results about {query}"


tools = [get_weather, calculate, search_web]

In [3]:
from langchain.agents.middleware import wrap_tool_call


@wrap_tool_call
def handle_tool_errors(request, handler):
    """Handle tool execution errors with custom messages."""
    try:
        return handler(request)
    except Exception as e:
        # Return a custom error message to the model
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"],
        )

In [11]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="gpt-4.1-nano",
    max_completion_tokens=2048,
    timeout=30,
)
agent = create_agent(
    model,
    tools=tools,
    middleware=[handle_tool_errors],
    system_prompt="You are a helpful assistant. Be concise and accurate.",
)


# Response output

- AIMessage
- AIMessageChunk (stream)
  - Source
  - Suggestion
- ToolMessage

- UserMessage

## Output schema



# Experiments with models

## get text content

In [5]:
# normal text content, no streaming
model_with_tools = model.bind_tools(tools)
result = model_with_tools.invoke("What is the weather in New York?")

In [6]:
from pprint import pprint
# from langchain_core.messages import message_to_dict
# message = message_to_dict(result)

message = result.model_dump()

pprint(message)

{'additional_kwargs': {'refusal': None},
 'content': '',
 'id': 'lc_run--019b36ad-8e71-70b0-946a-a3720d78a83e-0',
 'invalid_tool_calls': [],
 'name': None,
 'response_metadata': {'finish_reason': 'tool_calls',
                       'id': 'chatcmpl-CoUIaKSulPN8xOm5bhYTfxSAqOOB8',
                       'logprobs': None,
                       'model_name': 'gpt-5-nano-2025-08-07',
                       'model_provider': 'openai',
                       'service_tier': 'default',
                       'system_fingerprint': None,
                       'token_usage': {'completion_tokens': 88,
                                       'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                                                     'audio_tokens': 0,
                                                                     'reasoning_tokens': 64,
                                                                     'rejected_prediction_tokens': 0},
              

In [13]:
response = model.invoke("Why do parrots have colorful feathers?")
reasoning_steps = [b for b in response.content_blocks if b["type"] == "reasoning"]

In [48]:
response.model_dump()

{'content': '',
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 1048,
   'prompt_tokens': 14,
   'total_tokens': 1062,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 1048,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_provider': 'openai',
  'model_name': 'gpt-5-nano-2025-08-07',
  'system_fingerprint': None,
  'id': 'chatcmpl-CmK4e8eADr6pJFzylDyPn19SnuLk5',
  'service_tier': 'default',
  'finish_reason': 'length',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'lc_run--019b17ec-a79b-7bc0-802f-d96a666a2740-0',
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 14,
  'output_tokens': 1048,
  'total_tokens': 1062,
  'input_token_details': {'audio': 0, 'cache_read': 0},
  'output_token_details': {'audio': 0, 'reasoning': 1048}}}

In [ ]:
from langchain.tools import tool


@tool
def get_weather(location: str) -> str:
    """Get the weather at a location."""
    return f"It's sunny in {location}."


model_with_tools = model.bind_tools([get_weather])

response = model_with_tools.invoke("What's the weather like in Boston?")
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

Tool: get_weather
Args: {'location': 'Boston'}


# Experiments with agent

## output schema
content,
reasoning tokens,
tool calls,
tool fails
source url
source inline
usage data

```python
class UIMessageChunk:
  

```

In [14]:
agent_response = agent.invoke(
    {
        "messages": [
            HumanMessage(
                content="""
            Question 1: What is the weather in New York? 
            Question 2: Use reasoning to answer this. Give me the smallest positive integer that has exactly 10 divisors.
            Show every deduction you make and justify why your answer is the smallest."""
            )
        ]
    }
)

In [16]:
agent_response.get("messages")

[HumanMessage(content='\n            Question 1: What is the weather in New York? \n            Question 2: Use reasoning to answer this. Give me the smallest positive integer that has exactly 10 divisors.\n            Show every deduction you make and justify why your answer is the smallest.', additional_kwargs={}, response_metadata={}, id='d9735949-e088-4238-aecd-5b7d01b39e5b'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 792, 'prompt_tokens': 232, 'total_tokens': 1024, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 768, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CmjR6wTd8ytvRr8W2aTIUevQwoRtO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b1dbc-3e

In [18]:
agent_response_stream = agent.stream(
    {
        "messages": [
            HumanMessage(
                content="What is the weather in New York?, Sơn đến sớm hơn Hương, nhưng muộn hơn Ngọc. Ai là người đến sớm nhất?"
            )
        ]
    }
)

for chunk in agent_response_stream:
    print(chunk)

{'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 629, 'prompt_tokens': 208, 'total_tokens': 837, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CmjSkNg7Tcg8Y0RrUz2OHBvMq28LH', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b1dbd-d08f-7f51-bdb8-ed1e1213c151-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'New York'}, 'id': 'call_TTQ2v2DJTYUJCP7aDZ6xaWnk', 'type': 'tool_call'}, {'name': 'calculate', 'args': {'expression': '0'}, 'id': 'call_jQmeY8KzRsDZcp9fBwuBus7W', 'type': 'tool_call'}], usage_metadata={'input_tokens': 208, 'output_tokens': 629, 'total_tokens': 837, 'input_t

In [ ]:
for chunk in agent.stream(
    {
        "messages": [
            HumanMessage(content="What is the weather in New York?, What is 3^3?")
        ]
    }
):
    pprint(f"CHUNK: ==> {chunk}")
    for step, data in chunk.items():
        print("========================================================")
        print(f"step: {step}")
        pprint(f"content_blocks: {data['messages'][-1].content_blocks}")
        print(f"content: {chunk}")
    print("\n\n")

("CHUNK: ==> {'model': {'messages': [AIMessage(content='', "
 "additional_kwargs={'refusal': None}, response_metadata={'token_usage': "
 "{'completion_tokens': 46, 'prompt_tokens': 108, 'total_tokens': 154, "
 "'completion_tokens_details': {'accepted_prediction_tokens': 0, "
 "'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, "
 "'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, "
 "'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', "
 "'system_fingerprint': 'fp_7f8eb7d1f9', 'id': "
 "'chatcmpl-CoUpkiqCwgLkPtJvoOjENeOzJ5J4B', 'service_tier': 'default', "
 "'finish_reason': 'tool_calls', 'logprobs': None}, "
 "id='lc_run--019b36cc-f270-7dc0-9190-39e76fd9f3c2-0', tool_calls=[{'name': "
 "'get_weather', 'args': {'location': 'New York'}, 'id': "
 "'call_Ca1Nvra1wUgaoTLEAsSZ49Pr', 'type': 'tool_call'}, {'name': 'calculate', "
 "'args': {'expression': '3^3'}, 'id': 'call_c92cdvUWRBeTISmMYvcNQ3Tb', "
 "'type': 'tool_call'}], usage_m

: 

In [6]:
from langchain.agents import create_agent


def get_weather(city: str) -> str:
    """Get weather for a given city."""

    return f"It's always sunny in {city}!"


agent = create_agent(
    model="gpt-5-nano",
    tools=[get_weather],
)
async for token, metadata in agent.astream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="messages",
):
    print(f"node: {metadata['langgraph_node']}")
    print(f"content: {token.content_blocks}")
    print("\n")

node: model
content: [{'type': 'tool_call_chunk', 'id': 'call_4usR76tHxDfQ441xzR4seB07', 'name': 'get_weather', 'args': '', 'index': 0}]


node: model
content: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': '{"', 'index': 0}]


node: model
content: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': 'city', 'index': 0}]


node: model
content: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': '":"', 'index': 0}]


node: model
content: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': 'San', 'index': 0}]


node: model
content: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': ' Francisco', 'index': 0}]


node: model
content: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': '"}', 'index': 0}]


node: model
content: []


node: model
content: []


node: model
content: []


node: tools
content: [{'type': 'text', 'text': "It's always sunny in San Francisco!"}]


node: model
content: []


node: model
content: [{'ty

Agent: Search for AI news and summarize the findings
Agent: I don’t have live web access, so I can’t run a real-time search of AI news. I can still help in a few ways:

- Provide a concise, up-to-date-style briefing based on my latest training (through mid-2024) that highlights major themes and takeaways.
- Help you set up a process to get current AI news (suggest sources, RSS, or a simple script to fetch headlines from a news API) and then summarize whatever headlines you provide.
- If you paste specific headlines or links, I’ll read them and give you a structured summary and context.

Here’s a high-level summary of major AI news themes observed up to mid-2024 (without specific dates):
- Regulation and safety: Increased focus on AI risk management, transparency, accountability, and governance frameworks across regions (e.g., debates around AI Act-style rules, safety standards, and compliance requirements).
- Model development and safety: Ongoing advances in large language models and m

## Clean the code

- AIMessage
- AIMessageChunk (stream)
  - Source
  - Suggestion
- ToolMessage

- UserMessage

# Experiments with multi-agents